<a href="https://colab.research.google.com/github/pymc-devs/pytensor-workshop/blob/main/notebooks/exercises/pagerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**💡 To better engage gray mass we suggest you turn off Colab AI autocompletion in `Tools > Settings > AI Assistance`**

In [1]:
%%capture
try:
    import pytensor_workshop
except ModuleNotFoundError:
    !pip install git+https://github.com/pymc-devs/pytensor-workshop.git

In [2]:
import numpy as np
import pytensor
import pytensor.tensor as pt
from pytensor.graph import Variable
from pytensor.compile.function.types import Function

In [3]:
from pytensor_workshop import test

In [4]:
def dprint_with_preamble(fn, **kwargs):
    print("\n", "#"*24, sep="")
    print("Compiled function dprint")
    print("#"*24, "\n", sep="")
    fn.dprint(**kwargs)

# Pagerank algorithm with PyTensor

## Outline
We will use PyTensor to implement the PageRank algorithm on a real dataset: cross-refences from the very-much neglected PyTensor documentation. 

**Disclaimer: Nothing about this exercise really demands the use of a library like PyTensor. The point is to get familiar for those cases where it does make sense to use it!**

## Loading data

In [5]:
def load_M(normalize: bool = False):
    datafile = r"../../data/docs_cross_reference_matrix.txt"
    with open(datafile, "r") as f:
        pages = f.readline().strip("# ").strip('\n').split(",")
    M = np.loadtxt(datafile)
    if normalize:
        with np.errstate(all="ignore"):
            M /= M.sum(0)
        M = np.nan_to_num(M, nan=1/M.shape[0])
        return M
    return M, pages

M, pages = load_M()

In [6]:
len(pages), pages[:8]

(80,
 ['acknowledgement.rst',
  'adding.rst',
  'aliasing.rst',
  'basic.rst',
  'basic_opt.rst',
  'broadcasting.rst',
  'conditions.rst',
  'config.rst'])

In [7]:
M[:8, :8]

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

It's a pretty sparse matrix. It contains the number of references from one docs page to another.

The 1 in the second row, fourth column, corresponds to [this reference](https://github.com/pymc-devs/pytensor/blob/911c6a33c2bea6bf1d5b628154e84c43cbed1c63/doc/tutorial/adding.rst?plain=1#L199) from the `adding.rst` page to [this section](https://github.com/pymc-devs/pytensor/blob/911c6a33c2bea6bf1d5b628154e84c43cbed1c63/doc/library/tensor/basic.rst?plain=1#L1558) of the `basic.rst` page.

The other 1 in the fourth row, sixth column, is [this reference](https://github.com/pymc-devs/pytensor/blob/911c6a33c2bea6bf1d5b628154e84c43cbed1c63/doc/library/tensor/basic.rst?plain=1#L1560C57-L1560C79) from `basic.rst` to [this section](https://github.com/pymc-devs/pytensor/blob/911c6a33c2bea6bf1d5b628154e84c43cbed1c63/doc/tutorial/broadcasting.rst?plain=1#L7) in `tutbroadcasting.rst`.

The [PageRank algorithm](https://en.wikipedia.org/wiki/PageRank) is a way to rank pages based on how likely a user doing a random walk through links would end up in a given page in the long-run, assuming an equal probability of starting anywhere, and an equal probability of following any of the links available in the current page to subsequent pages. It can be implemnted using [power iteration](https://en.wikipedia.org/wiki/Power_iteration), which is a fancy way of incrementally finding the highest eigenvalue/vector of a matrix.

The highest eigenvector of a markov chain according to a transition probability matrix gives to the stationary/long-term distribution ([or something like that](https://en.wikipedia.org/wiki/Markov_chain#Stationary_distribution_relation_to_eigenvectors_and_simplices)).

We will apply this to rank the PyTensor documentation pages, based on internal link counts encoded in the matrix above. The first step is to rescale the out-going links of a page, so they correspond to probabilities. For pages that have no out-going links, we will assume users are equally likely to go to any other page. Here is a dirty way to get it:

In [8]:
Mp = M / M.sum(axis=0)
Mp = np.nan_to_num(Mp, nan=1/M.shape[0])
Mp[:8, :8]

/tmp/ipykernel_146345/3983240304.py:1: RuntimeWarning: invalid value encountered in divide
  Mp = M / M.sum(axis=0)


array([[0.0125, 0.0125, 0.    , 0.    , 0.0125, 0.    , 0.0125, 0.    ],
       [0.0125, 0.0125, 0.    , 0.25  , 0.0125, 0.    , 0.0125, 0.    ],
       [0.0125, 0.0125, 0.    , 0.    , 0.0125, 0.    , 0.0125, 0.    ],
       [0.0125, 0.0125, 0.    , 0.    , 0.0125, 0.5   , 0.0125, 0.    ],
       [0.0125, 0.0125, 0.    , 0.    , 0.0125, 0.    , 0.0125, 0.    ],
       [0.0125, 0.0125, 0.    , 0.    , 0.0125, 0.    , 0.0125, 0.    ],
       [0.0125, 0.0125, 0.    , 0.    , 0.0125, 0.    , 0.0125, 0.    ],
       [0.0125, 0.0125, 0.    , 0.    , 0.0125, 0.    , 0.0125, 0.    ]])

## Exercise 1: Redo the normalization using a PyTensor function

As a (very silly) warmup exercise, let's do the matrix normalization we just computed above using PyTensor operations.

The following test function expects a compiled PyTensor function should accept a square matrix M (of any size) as input and output the rescaled Mp with the special behavior for empty columns.

**We will call `dprint` (short for debug print) on the compiled function. Try and compare that with the original PyTensor graph before compilation, by calling `dprint` on the variable that is used as the output of the function**

https://pytensor.readthedocs.io/en/latest/tutorial/printing_drawing.html#debug-print

In [9]:
M = pt.matrix("M")
Mp = ...

# Mp.dprint()  
# Mp_fn = pytensor.function([M], Mp) 


@test
def test_Mp_function(fn):
    assert isinstance(fn, Function)    
    
    dprint_with_preamble(fn)

    M, _ = load_M()
    Mp = load_M(normalize=True)
    
    np.testing.assert_allclose(fn(M), Mp)

    # Sneaky test that shapes weren't hardcoded
    try:
        smaller_M_res = fn(M[1:, 1:])
    except ValueError as exc:
        raise ValueError("Function failed to evaluate with smaller matrix!") from exc
    with np.errstate(all="ignore"):
        expected_res = np.nan_to_num(M[1:, 1:] / M[1:, 1:].sum(0), nan=1/(M.shape[0]-1))
    np.testing.assert_allclose(smaller_M_res, expected_res)
    
# test_Mp_function(Mp_fn)  # Uncomment me

Can you make sense of the computational graph before and after compilation?

## Exercise 2: Using eval for a single time computation

Compiling a PyTensor function for a single time use is as silly as using the [is-odd](https://www.npmjs.com/package/is-odd) package. 
For one time use, or just debugging intermediate values, there's a `.eval()` method that can be used instead. It requires specifying any inputs and values as, respectively, the keys and values of a dictionary. 

This exercise asks you to pass the output variable and the dictionary inputs needed to to this one time evaluation

In [10]:
var = ...
eval_dict = ...

@test
def test_Mp_eval(variable, eval_dict):
    assert isinstance(variable, Variable)
    assert isinstance(eval_dict, dict)

    Mp = load_M(normalize=True)
    np.testing.assert_allclose(variable.eval(eval_dict), Mp)

# test_Mp_eval(var, eval_dict)  # Uncomment me

## Exercise 3: PageRank step function

Let's take a tiny step into a realistic use of PyTensor by compiling a function that will be evaluated more than once!
This exercise asks you to define a function that performs one step of the PageRank iterative algorithm. We'll start simple and **NOT** include a [damping factor](https://en.wikipedia.org/wiki/PageRank#Damping_factor).

The step function should compute: $v_{t+1} = M \cdot v_t$, where v corresponds to the stationary probability distribution over pages. 
We'll start it as 1/N

Once again, the function should be agnostic to the size of the Matrix! This will be tested.

Try to compare and make sense of the uncompiled and compiled graphs

In [11]:
M = ...
v = ...
next_v = ...

# next_v.dprint()
# step_fn = pytensor.function([M, v], next_v)

@test
def test_step_function(step_fn):
    assert isinstance(step_fn, Function)    

    dprint_with_preamble(step_fn)
    
    Mp = load_M(normalize=True)
    v = np.ones(Mp.shape[0]) / Mp.shape[0]
    # Apply step function 100 times
    for i in range(100):
        v = step_fn(Mp, v)

    assert np.isclose(v.sum(), 1.0),  "Does not look like a probability vector"
    np.testing.assert_allclose(
        np.sort(v)[-3:],
        [0.25430864, 0.25443471, 0.47704864],
        err_msg="Results do not match expectation!",
    )
  

    # Sneaky test that shapes weren't hardcoded
    try:
        smaller_res = step_fn(Mp[1:, 1:], v[1:])
    except ValueError as exc:
        raise ValueError("Function failed to evaluate with smaller vector and matrix!") from exc
    assert smaller_res.shape == (Mp.shape[0] - 1,)

# test_step_function(step_fn)  # Uncomment me

## Open ended exercise: Investigate the results

This is not a PyTensor exercise, but let's sort the rank of the pages we got.

Find out what are the highest ranking pages, and what is their stationary probability. Does this seem reasonable?

In [12]:
...

Ellipsis

## Exercise 4: Damping factor

The proper algorithm of PageRank includes a damping factor which according to [Wikipedia](https://en.wikipedia.org/wiki/PageRank#Damping_factor):

> The PageRank theory holds that an imaginary surfer who is randomly clicking on links will eventually stop clicking. The probability, at any step, that the person will continue following links is a damping factor d. The probability that they instead jump to any random page is 1 - d. Various studies have tested different damping factors, but it is generally assumed that the damping factor will be set around 0.85.

For this next exercise extend the PyTensor step function to include a scalar damping parameter `d`

$$ v_{t+1} = d M \cdot v_t + \frac{1 - d}{N} $$

In [13]:
d = ...
v = ...
M = ...
next_v = ...

# next_v.dprint()
# step_fn = pytensor.function([d, M, v], next_v)

@test
def test_damp_step_function(step_fn):
    assert isinstance(step_fn, Function)    

    dprint_with_preamble(step_fn)
    
    Mp = load_M(normalize=True)

    # With d=1 we should get the same results as before
    v = np.ones(Mp.shape[0]) / Mp.shape[0]
    for i in range(100):
        v = step_fn(1.0, Mp, v)

    assert np.isclose(v.sum(), 1.0),  "Does not look like a probability vector"
    np.testing.assert_allclose(
        np.sort(v)[-3:],
        [0.25430864, 0.25443471, 0.47704864],
        err_msg="Results do not match old results with d=1",
    )

    # With d=.85 we should get something else
    v = np.ones(Mp.shape[0]) / Mp.shape[0]
    for i in range(100):
        v = step_fn(0.85, Mp, v)
    assert np.isclose(v.sum(), 1.0),  "Does not look like a probability vector"
    np.testing.assert_allclose(
        np.sort(v)[-3:],
        [0.06142576, 0.0773205 , 0.11942332],
        err_msg="Results do not match old results with d=1",
    )
  

    # Sneaky test that shapes weren't hardcoded
    try:
        smaller_res = step_fn(1.0, Mp[1:, 1:], v[1:])
    except ValueError as exc:
        raise ValueError("Function failed to evaluate with smaller vector and matrix!") from exc
    assert smaller_res.shape == (Mp.shape[0] - 1,)

# test_damp_step_function(step_fn)  # Uncomment me

## Open ended exercise: Investigate the results

Check the new results when `d=0.85`. Does it look more reasonable? What pages (if any) seem to have an inflated pagerank? Perhaps they hired a CEO master to get ahead!

In [14]:
...

Ellipsis

## Exercise 5: Multiple steps in one go

PyTensor allows us to escape the slow-land of Python, but we are not doing that to a great extent so far.
For this next exercise let's define a larger function, that does 5 steps at a time instead of only 1.
The output should be equvalent to running the inner loop you saw in the previous tests 5 times.

Again try and look at the before and after compilation graphs. Was it what you would have expected?
Do you see a problem if we try to scale this so that the whole algorithm runs in a single PyTensor function call?
How could we get around that?

In [15]:
d = ...
v = ...
M = ...
next_v = ...

# next_v.dprint()
# step_fn = pytensor.function([d, M, v], next_v)


@test
def test_damp_5step_function(step_fn):
    assert isinstance(step_fn, Function)    

    dprint_with_preamble(step_fn)
    
    Mp = load_M(normalize=True)

    # With 20 steps we should get the same results as before
    v = np.ones(Mp.shape[0]) / Mp.shape[0]
    for i in range(20):
        v = step_fn(1.0, Mp, v)

    assert np.isclose(v.sum(), 1.0),  "Does not look like a probability vector"
    np.testing.assert_allclose(
        np.sort(v)[-3:],
        [0.25430864, 0.25443471, 0.47704864],
        err_msg="Results do not match old results with d=1",
    )

    # With d=.85 we should get something else
    v = np.ones(Mp.shape[0]) / Mp.shape[0]
    for i in range(20):
        v = step_fn(0.85, Mp, v)
    assert np.isclose(v.sum(), 1.0),  "Does not look like a probability vector"
    np.testing.assert_allclose(
        np.sort(v)[-3:],
        [0.06142576, 0.0773205 , 0.11942332],
        err_msg="Results do not match old results with d=1",
    )
  
    # Sneaky test that shapes weren't hardcoded
    try:
        smaller_res = step_fn(1.0, Mp[1:, 1:], v[1:])
    except ValueError as exc:
        raise ValueError("Function failed to evaluate with smaller vector and matrix!") from exc
    assert smaller_res.shape == (Mp.shape[0] - 1,)

# test_damp_5step_function(step_fn)  # Uncomment me

## Exercise 6: vectorize single step function on damp factor

Say we want to test sensitivity to the damping factor. We could run the same algorithm multiple times for different values of d, but that grows linearly with the number of values we want to test.

Instead we may want to batch our computational graph so we can run multiple scenarios concurrently.

To that end let's define a batched step function that allows us to update 3 page rank vectors associated with 3 distinct damping factors.

$$ v_{i, t+1} = d_iM \cdot v_{i, t}$$

We are back to a **single** step for this exercise!

**Note the use of `print_type=True` in this exercise. It can be useful to make sense of shapes!**

In [16]:
ds = pt.vector("d", shape=(3,))
vs = pt.matrix("v", shape=(3, None,))
M = pt.matrix("M", shape=(None, None))

next_vs = ...
# next_vs.dprint(print_type=True)
# step_fn = pytensor.function([ds, M, vs], next_vs)

@test
def test_batch_damp_step_function(step_fn):
    assert isinstance(step_fn, Function)    

    dprint_with_preamble(step_fn, print_type=True)
    
    Mp = load_M(normalize=True)

    vs = np.ones((3, Mp.shape[0])) / Mp.shape[0]
    for i in range(100):
        vs = step_fn([1.0, 0.85, 0.5], Mp, vs)

    np.testing.assert_allclose(
        vs.sum(axis=-1), 
        1.0, 
        err_msg="Does not look like a probability vector",
    )

    # The first v with d=1 should look like before
    np.testing.assert_allclose(
        np.sort(vs[0])[-3:],
        [0.25430864, 0.25443471, 0.47704864],
        err_msg="Results do not match old results with d=1",
    )

    # With d=.85 we should get something else
    np.testing.assert_allclose(
        np.sort(vs[1])[-3:],
        [0.06142576, 0.0773205 , 0.11942332],
        err_msg="Results do not match old results with d=1",
    )

# test_batch_damp_step_function(step_fn)  # Uncomment me

## Exercise 7: Vectorization the easy way

Use `vectorize_graph` to build the same function as above automatically from the single case

In [17]:
from pytensor.graph.replace import vectorize_graph

https://pytensor.readthedocs.io/en/latest/library/graph/replace.html#pytensor.graph.replace.vectorize_graph

In [18]:
d = pt.scalar("d")
v = pt.vector("v")
M = pt.matrix("M")

next_v = ...

ds = ...
vs = ...
# next_vs = vectorize_graph(v, ...)

# next_vs.dprint(print_type=True)
# step_fn = pytensor.function([ds, M, vs], next_vs)

# test_batch_damp_step_function(step_fn)  # uncomment me

## Exercise 8: More constrained graph

Redefine the single step damp function using static shape inputs. such as `x = pt.vector(shape=(9,)` and a constant `d=0.85`. 
Compare the compiled function with the one that allows dynamic shapes and a variable d. 

Which one looks more performant? When would you prefer one or the other?

In [19]:
d = ...
v = ...
M = ...

next_v = ...

# next_v.dprint(print_type=True)
# step_fn = pytensor.function([M, v], next_v)

@test
def test_constrained_damp_step_function(step_fn):
    assert isinstance(step_fn, Function)    

    dprint_with_preamble(step_fn, print_type=True)
    
    Mp = load_M(normalize=True)

    v = np.ones(Mp.shape[0]) / Mp.shape[0]
    for i in range(100):
        v = step_fn(Mp, v)
    
    assert np.isclose(v.sum(), 1.0),  "Does not look like a probability vector"
    np.testing.assert_allclose(
        np.sort(v)[-3:],
        [0.06142576, 0.0773205 , 0.11942332],
        err_msg="Results do not match old results with d=1",
    )
  

    # Sneaky test that shapes weren't hardcoded
    try:
        smaller_res = step_fn(Mp[1:, 1:], v[1:])
    except TypeError:
        # This is the right behavior
        return
    else:
        raise ValueError("Function should have failed to evaluate with smaller vector and matrix!")

# test_constrained_damp_step_function(step_fn)  # Uncomment me

## Open-ended exercise: Use clone_replace to define constrained function from unconstrained one

Last exercise! Given a predefined computational graph of the step function that is unconstrained (dynamic shapes, and variable damping factor), provide a set of replacements that when passed to `clone_replace` will automatically redefine the constrained graph.

In [20]:
from pytensor.graph.replace import graph_replace

https://pytensor.readthedocs.io/en/latest/library/graph/replace.html#pytensor.graph.replace.graph_replace

In [21]:
...

Ellipsis